In [24]:
%load_ext autoreload
%autoreload 2
import os
import sys
import torch
import numpy as np
import pandas as pd
import time
import argparse
from functools import partial

# add code directory to path
import sys
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from metrics import top_k_onehot_indicator
from torch_perturb.perturbations import perturbed
from torch_models import NegativeBinomialDebug, torch_bpr_uncurried, deterministic_bpr


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
'''
from torch.distributions import NegativeBinomial
from torch import nn
def single_tensor_to_params(param_size_shape_list, single_tensor):
        params = []
        idx = 0
        for (numel, shape) in param_size_shape_list:
            num_elements = numel
            param_data = single_tensor[idx:idx+num_elements].view(shape)
            params.append(param_data)
            idx += num_elements
        return params

def build_from_single_tensor(single_tensor, param_size_shape_list, X, time):
    beta_0, beta, b_0, b_1, log_sigma_0, log_sigma_1, rho, siginv_theta = single_tensor_to_params(param_size_shape_list, single_tensor)
    fixed_effects = beta_0 + torch.einsum('tli,i->tl', X, beta)
    random_intercepts = b_0.expand(X.shape[0], -1)
    random_slopes = b_1.expand(X.shape[0], -1)
    
    log_mu = fixed_effects + random_intercepts + random_slopes * time

    # Use softplus to ensure mu is positive and grows more slowly
    mu = nn.functional.softplus(log_mu)

    # Calculate theta probability
    theta = torch.nn.functional.sigmoid(siginv_theta)

    
    return NegativeBinomial(total_count=mu, probs=theta)
'''

"\nfrom torch.distributions import NegativeBinomial\nfrom torch import nn\ndef single_tensor_to_params(param_size_shape_list, single_tensor):\n        params = []\n        idx = 0\n        for (numel, shape) in param_size_shape_list:\n            num_elements = numel\n            param_data = single_tensor[idx:idx+num_elements].view(shape)\n            params.append(param_data)\n            idx += num_elements\n        return params\n\ndef build_from_single_tensor(single_tensor, param_size_shape_list, X, time):\n    beta_0, beta, b_0, b_1, log_sigma_0, log_sigma_1, rho, siginv_theta = single_tensor_to_params(param_size_shape_list, single_tensor)\n    fixed_effects = beta_0 + torch.einsum('tli,i->tl', X, beta)\n    random_intercepts = b_0.expand(X.shape[0], -1)\n    random_slopes = b_1.expand(X.shape[0], -1)\n    \n    log_mu = fixed_effects + random_intercepts + random_slopes * time\n\n    # Use softplus to ensure mu is positive and grows more slowly\n    mu = nn.functional.softplus(lo

In [26]:
def convert_df_to_3d_array(df):
    # Ensure the DataFrame has a MultiIndex with 'geoid' and 'timestep'
    if not isinstance(df.index, pd.MultiIndex) or set(df.index.names) != {'geoid', 'timestep'}:
        raise ValueError("DataFrame must have a MultiIndex with levels 'geoid' and 'timestep'")

    # Get unique geoids and timesteps, sorted
    geoids = sorted(df.index.get_level_values('geoid').unique())
    timesteps = sorted(df.index.get_level_values('timestep').unique())

    # Create a mapping of geoids to indices
    geoid_to_idx = {geoid: idx for idx, geoid in enumerate(geoids)}

    # Initialize the 3D array
    num_timesteps = len(timesteps)
    num_locations = len(geoids)
    num_features = len(df.columns)
    X = np.zeros((num_timesteps, num_locations, num_features))

    # Fill the 3D array
    for (geoid, timestep), row in df.iterrows():
        t_idx = timesteps.index(timestep)
        g_idx = geoid_to_idx[geoid]
        X[t_idx, g_idx, :] = row.values

    return X, geoids, timesteps

def convert_y_df_to_2d_array(y_df, geoids, timesteps):
    # Ensure the DataFrame has a MultiIndex with 'geoid' and 'timestep'
    if not isinstance(y_df.index, pd.MultiIndex) or set(y_df.index.names) != {'geoid', 'timestep'}:
        raise ValueError("DataFrame must have a MultiIndex with levels 'geoid' and 'timestep'")

    # Initialize the 2D array
    num_timesteps = len(timesteps)
    num_locations = len(geoids)
    y = np.zeros((num_timesteps, num_locations))

    # Create a mapping of geoids to indices
    geoid_to_idx = {geoid: idx for idx, geoid in enumerate(geoids)}

    # Fill the 2D array
    for (geoid, timestep), value in y_df.iloc[:, 0].items():
        t_idx = timesteps.index(timestep)
        g_idx = geoid_to_idx[geoid]
        y[t_idx, g_idx] = value

    return y

def evaluate_model(model, X, y, time, K, M_score_func, perturbed_top_K_func):
    """Evaluate model on given data and return metrics."""
    with torch.no_grad():
        dist = model(X, time)
        
        # Sample and calculate ratio ratings
        y_sample_TMS = dist.sample((M_score_func,)).permute(1, 0, 2)
        ratio_rating_TMS = y_sample_TMS/(1+y_sample_TMS.sum(dim=-1, keepdim=True))
        ratio_rating_TS = ratio_rating_TMS.mean(dim=1)
        
        # Calculate metrics
        nll = -model.log_likelihood(y, X, time)
        perturbed_bpr_T = torch_bpr_uncurried(ratio_rating_TS, y, K=K, 
                                             perturbed_top_K_func=perturbed_top_K_func)
        deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, y, K=K)
        
        metrics = {
            'nll': nll.item(),
            'perturbed_bpr': torch.mean(perturbed_bpr_T).item(),
            'deterministic_bpr': torch.mean(deterministic_bpr_T).item()
        }
        
        return metrics

def train_epoch_neg_binom(model, optimizer, K, threshold,
                         M_score_func, feat_TSF,
                         time_T, train_y_TS,
                         perturbed_top_K_func, bpr_weight, nll_weight, update=True):
    """Train one epoch of the negative binomial model."""
    model.train()
    optimizer.zero_grad()
    
    total_loss = 0
    total_gradient_P = None
    if (nll_weight > 0) and (bpr_weight==0) and (False):
        nll = -model.log_likelihood(train_y_TS, feat_TSF, time_T)
        nll.backward()
        if update: 
            optimizer.step()
        metrics = {
            'loss': nll.item(),
            'deterministic_bpr': 0,
            'perturbed_bpr': 0,
            'nll': nll.item()}
    else:
        for t in range(feat_TSF.shape[0]):
            dist = model(feat_TSF[t:t+1], time_T[t:t+1])

            y_sample_TMS = dist.sample((M_score_func,)).permute(1, 0, 2)
            action_denominator_TM = y_sample_TMS.sum(dim=-1, keepdim=True) + 1 

            ratio_rating_TMS = y_sample_TMS / action_denominator_TM
            ratio_rating_TS = ratio_rating_TMS.mean(dim=1)
            ratio_rating_TS.requires_grad_(True)

            def get_log_probs_baked(param, y_samples_chunk):
                distribution = model.build_from_single_tensor(param, feat_TSF[t:t+1], time_T[t:t+1])
                log_probs_chunk = distribution.log_prob(y_samples_chunk.permute(1, 0, 2)).permute(1, 0, 2)
                return log_probs_chunk

            # Define the chunk size
            chunk_size = 1  # Adjust this to fit your memory constraints
            jac_TMSP_chunks = []

            # Loop through chunks of y_sample_TMS
            for i in range(0, M_score_func, chunk_size):
                print(f'Chunky: {i}')
                y_samples_chunk = y_sample_TMS[:, i:i+chunk_size, :]
                jac_chunk = torch.autograd.functional.jacobian(
                    lambda param: get_log_probs_baked(param, y_samples_chunk),
                    model.params_to_single_tensor(),
                    strategy='forward-mode',
                    vectorize=True
                )
                jac_TMSP_chunks.append(jac_chunk)

            # Concatenate all chunks along the sample dimension
            jac_TMSP = torch.cat(jac_TMSP_chunks, dim=1)  # Adjust dim if needed

            score_func_estimator_TMSP = jac_TMSP * ratio_rating_TMS.unsqueeze(-1)
            score_func_estimator_TSP = score_func_estimator_TMSP.mean(dim=1)    

            positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS[t:t+1]), 
                                                K=K, perturbed_top_K_func=perturbed_top_K_func)

            if nll_weight > 0:
                bpr_threshold_diff_T = positive_bpr_T - threshold
                violate_threshold_flag = bpr_threshold_diff_T < 0
                negative_bpr_loss = torch.mean(-bpr_threshold_diff_T * violate_threshold_flag)
            else:
                negative_bpr_loss = torch.mean(-positive_bpr_T)

            nll = -model.log_likelihood(train_y_TS[t:t+1], feat_TSF[t:t+1], time_T[t:t+1])
            loss = bpr_weight * negative_bpr_loss + nll_weight * nll
            loss.backward()

            loss_grad_TS = ratio_rating_TS.grad
            gradient_TSP = score_func_estimator_TSP * torch.unsqueeze(loss_grad_TS, -1)
            gradient_P = torch.sum(gradient_TSP, dim=[0, 1])
            
            if total_gradient_P is None:
                total_gradient_P = gradient_P
            else:
                total_gradient_P += gradient_P
            
            total_loss += loss.item()

        gradient_tuple = model.single_tensor_to_params(total_gradient_P)

        for param, gradient in zip(model.parameters(), gradient_tuple):
            if nll_weight > 0:
                gradient = gradient + param.grad
            param.grad = gradient

        if update:
            optimizer.step()

        deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, torch.tensor(train_y_TS), K=K)
        det_bpr = torch.mean(deterministic_bpr_T)

        metrics = {
            'loss': total_loss ,
            'deterministic_bpr': det_bpr.item(),
            'perturbed_bpr': torch.mean(positive_bpr_T).item(),
            'nll': nll.item()
        }

    return metrics, model

def main(K=None, step_size=None, epochs=None, bpr_weight=None,
         nll_weight=None, seed=None, outdir=None, threshold=None,
         perturbed_noise=None, num_score_samples=None, num_pert_samples=None,
         data_dir=None, device='cuda', val_freq=10):
    """Main training loop with command line arguments."""
    
    # Set random seed for reproducibility
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    # Load training data
    train_X_df = pd.read_csv(os.path.join(data_dir, 'train_x.csv'), index_col=[0,1])
    train_Y_df = pd.read_csv(os.path.join(data_dir, 'train_y.csv'), index_col=[0,1])
    
    # Load validation data
    val_X_df = pd.read_csv(os.path.join(data_dir, 'valid_x.csv'), index_col=[0,1])
    val_Y_df = pd.read_csv(os.path.join(data_dir, 'valid_y.csv'), index_col=[0,1])
    
    # Process training data
    train_X, geoids, timesteps = convert_df_to_3d_array(train_X_df)#.drop(columns='timestep.1'))
    train_time_arr = np.array([timesteps] * len(geoids)).T
    train_y = convert_y_df_to_2d_array(train_Y_df, geoids, timesteps)

    # Process validation data
    val_X, _, val_timesteps = convert_df_to_3d_array(val_X_df)#.drop(columns='timestep.1'))
    val_time_arr = np.array([val_timesteps] * len(geoids)).T
    val_y = convert_y_df_to_2d_array(val_Y_df, geoids, val_timesteps)

    # Convert to tensors and move to device
    X_train = torch.tensor(train_X, dtype=torch.float32).to(device)
    y_train = torch.tensor(train_y, dtype=torch.float32).to(device)
    time_train = torch.tensor(train_time_arr, dtype=torch.float32).to(device)
    
    X_val = torch.tensor(val_X, dtype=torch.float32).to(device)
    y_val = torch.tensor(val_y, dtype=torch.float32).to(device)
    time_val = torch.tensor(val_time_arr, dtype=torch.float32).to(device)

    # Initialize model
    model = NegativeBinomialDebug(
        num_locations=len(geoids),
        num_fixed_effects=train_X.shape[2], device=device
    ).to(device)

    # Setup optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=step_size)

    # Setup top-k function
    top_k_func = partial(top_k_onehot_indicator, k=K)
    perturbed_top_K_func = perturbed(top_k_func, sigma=perturbed_noise, num_samples=num_pert_samples)

    # Initialize metric tracking with separate epoch tracking for validation
    metrics = {
        'train': {
            'epochs': [], 
            'loss': [], 
            'nll': [], 
            'perturbed_bpr': [], 
            'deterministic_bpr': []
        },
        'val': {
            'epochs': [], 
            'nll': [], 
            'perturbed_bpr': [], 
            'deterministic_bpr': []
        },
        'times': []
    }

    best_val_loss = float('inf')
    
    # Training loop
    for epoch in range(epochs):
        print(f'EPOCH: {epoch}')
        start = time.time()
        
        # Training step
        train_metrics, model = train_epoch_neg_binom(
            model, optimizer, K, threshold,
            num_score_samples, X_train, time_train,
            y_train, perturbed_top_K_func,
            bpr_weight, nll_weight, device
        )
        
        # Update training metrics
        metrics['train']['epochs'].append(epoch)
        for metric, value in train_metrics.items():
            metrics['train'][metric].append(value)
        
        # Validation step (every val_freq epochs)
        if epoch % val_freq == 0:
            model.eval()
            val_metrics = evaluate_model(
                model, X_val, y_val, time_val,
                K, num_score_samples, perturbed_top_K_func
            )
            
            # Update validation metrics
            metrics['val']['epochs'].append(epoch)
            for metric, value in val_metrics.items():
                metrics['val'][metric].append(value)
            
            # Save best model
            val_loss = val_metrics['nll'] * nll_weight
            if bpr_weight > 0:
                val_loss -= val_metrics['perturbed_bpr'] * bpr_weight
                
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                if not os.path.exists(outdir):
                    os.makedirs(outdir)
                torch.save(model.state_dict(), f'{outdir}/best_model.pth')
        
        end = time.time()
        metrics['times'].append(end - start)
        
        # Print progress
        print(f"Train - Loss: {train_metrics['loss']:.4f}, NLL: {train_metrics['nll']:.4f}, "
              f"BPR: {train_metrics['deterministic_bpr']:.4f}")
        if epoch % val_freq == 0:
            print(f"Val - NLL: {val_metrics['nll']:.4f}, "
                  f"BPR: {val_metrics['deterministic_bpr']:.4f}")
        
        # Save checkpoints
        if epoch % 100 == 0:
            if not os.path.exists(outdir):
                os.makedirs(outdir)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'metrics': metrics,
                'best_val_loss': best_val_loss
            }, f'{outdir}/checkpoint.pth')
            
            # Save metrics separately for easier analysis
            # Create DataFrames with proper indexing
            train_df = pd.DataFrame(metrics['train']).set_index('epochs')
            val_df = pd.DataFrame(metrics['val']).set_index('epochs')
            times_df = pd.DataFrame({'times': metrics['times']}, index=range(len(metrics['times'])))
            
            train_df.to_csv(f'{outdir}/train_metrics.csv')
            val_df.to_csv(f'{outdir}/val_metrics.csv')
            times_df.to_csv(f'{outdir}/time_metrics.csv')

In [35]:
K = 50
bpr_weight = 1
nll_weight = 1
step_size = 0.001
num_score_samples = 1
num_pert_samples = 100
seed = 123
perturbed_noise = 0.001
threshold = 0.5
epochs = 1
outdir = '/cluster/tufts/hugheslab/kheuto01/debug'
data_dir = '~/'
bird = True

device = 'cuda'
val_freq = 10

In [36]:
# Set random seed for reproducibility
if seed is not None:
    torch.manual_seed(seed)
    np.random.seed(seed)

if bird:
    bird_str = 'bird_'
else:
    bird_str = ''

# Load training data
train_X_df = pd.read_csv(os.path.join(data_dir, f'{bird_str}train_x.csv'), index_col=[0,1])
train_Y_df = pd.read_csv(os.path.join(data_dir, f'{bird_str}train_y.csv'), index_col=[0,1])

# Load validation data
val_X_df = pd.read_csv(os.path.join(data_dir, f'{bird_str}valid_x.csv'), index_col=[0,1])
val_Y_df = pd.read_csv(os.path.join(data_dir, f'{bird_str}valid_y.csv'), index_col=[0,1])

# Process training data
train_X, geoids, timesteps = convert_df_to_3d_array(train_X_df)#.drop(columns='timestep.1'))
train_time_arr = np.array([timesteps] * len(geoids)).T
train_y = convert_y_df_to_2d_array(train_Y_df, geoids, timesteps)

# Process validation data
val_X, _, val_timesteps = convert_df_to_3d_array(val_X_df)#.drop(columns='timestep.1'))
val_time_arr = np.array([val_timesteps] * len(geoids)).T
val_y = convert_y_df_to_2d_array(val_Y_df, geoids, val_timesteps)

# Convert to tensors and move to device
X_train = torch.tensor(train_X, dtype=torch.float32).to(device)
y_train = torch.tensor(train_y, dtype=torch.float32).to(device)
time_train = torch.tensor(train_time_arr, dtype=torch.float32).to(device)

X_val = torch.tensor(val_X, dtype=torch.float32).to(device)
y_val = torch.tensor(val_y, dtype=torch.float32).to(device)
time_val = torch.tensor(val_time_arr, dtype=torch.float32).to(device)

# Initialize model
model = NegativeBinomialDebug(
    num_locations=len(geoids),
    num_fixed_effects=train_X.shape[2], device=device
).to(device)

# Setup optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=step_size)

# Setup top-k function
top_k_func = partial(top_k_onehot_indicator, k=K)
perturbed_top_K_func = perturbed(top_k_func, sigma=perturbed_noise, num_samples=num_pert_samples)

# Initialize metric tracking with separate epoch tracking for validation
metrics = {
    'train': {
        'epochs': [], 
        'loss': [], 
        'nll': [], 
        'perturbed_bpr': [], 
        'deterministic_bpr': []
    },
    'val': {
        'epochs': [], 
        'nll': [], 
        'perturbed_bpr': [], 
        'deterministic_bpr': []
    },
    'times': []
}

best_val_loss = float('inf')


In [44]:
X_train.min()

tensor(-97.0381, device='cuda:0')

In [37]:
model.log_likelihood(y_train, X_train, time_train)

tensor(-inf, device='cuda:0', grad_fn=<AddBackward0>)

In [32]:
start = time.time()
epoch = 0

# Training step
train_metrics, model = train_epoch_neg_binom(
    model, optimizer, K, threshold,
    num_score_samples, X_train, time_train,
    y_train, perturbed_top_K_func,
    bpr_weight, nll_weight, update=False
)

# Update training metrics
metrics['train']['epochs'].append(epoch)
for metric, value in train_metrics.items():
    metrics['train'][metric].append(value)

Chunky: 0


/tmp/ipykernel_259927/284826383.py:130: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS[t:t+1]),


Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0
Chunky: 0


/tmp/ipykernel_259927/284826383.py:165: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, torch.tensor(train_y_TS), K=K)


In [33]:
param_list = [param.grad for param in model.parameters()]
param_list

[tensor([nan], device='cuda:0', grad_fn=<AddBackward0>),
 tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:0',
        grad_fn=<AddBackward0>),
 tensor([nan, nan, nan,  ..., nan, nan, nan], device='cuda:0',
        grad_fn=<AddBackward0>),
 tensor([nan, nan, nan,  ..., nan, nan, nan], device='cuda:0',
        grad_fn=<AddBackward0>),
 tensor([0.], device='cuda:0', grad_fn=<AddBackward0>),
 tensor([0.], device='cuda:0', grad_fn=<AddBackward0>),
 tensor([0.], device='cuda:0', grad_fn=<AddBackward0>),
 tensor([-1.6950], device='cuda:0', grad_fn=<AddBackward0>)]

In [34]:
metrics

{'train': {'epochs': [0],
  'loss': [inf],
  'nll': [inf],
  'perturbed_bpr': [0.024308940395712852],
  'deterministic_bpr': [0.0]},
 'val': {'epochs': [],
  'nll': [],
  'perturbed_bpr': [],
  'deterministic_bpr': []},
 'times': []}

In [8]:
if not os.path.exists(outdir):
    os.makedirs(outdir)
torch.save(model.state_dict(), f'{outdir}/best_model.pth')

In [9]:
# Initialize model with correct parameters
model2  = NegativeBinomialRegressionModel(
    num_locations=len(geoids),
    num_fixed_effects=train_X.shape[2], device=device
).to(device)

# Load saved weights
model2.load_state_dict(torch.load(f'{outdir}/best_model.pth', map_location=device))
model2.eval()

NameError: name 'NegativeBinomialRegressionModel' is not defined

In [18]:
model2.state_dict()['beta'].shape

torch.Size([14])

In [19]:
train_X.shape[2]

14

In [20]:
train_X_df

,,deaths_sp_lag,INTPTLAT,INTPTLON,timestep.1,svi_theme1,svi_theme2,svi_theme3,svi_theme4,svi_total_,prev_deaths_05back,prev_deaths_04back,prev_deaths_03back,prev_deaths_02back,prev_deaths_01back
geoid,timestep,,,,,,,,,,,,,,
17031010100,2,0.600000,42.021255,-87.669830,2,0.7266,0.3552,0.6652,0.8699,0.7562,0.0,0.0,0.0,0.0,1.0
17031010201,2,0.142857,42.016008,-87.680148,2,0.8108,0.5819,0.8880,0.9470,0.9325,0.0,0.0,0.0,0.0,0.0
17031010202,2,0.625000,42.016048,-87.673326,2,0.7016,0.3417,0.8568,0.9801,0.8619,0.0,0.0,0.0,0.0,0.0
17031010300,2,0.500000,42.015943,-87.666539,2,0.5695,0.1866,0.8379,0.9833,0.7600,0.0,0.0,0.0,0.0,2.0
17031010400,2,0.750000,42.006411,-87.658816,2,0.5516,0.0382,0.4726,0.9984,0.6209,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17031843700,5,0.307692,41.945560,-87.690034,5,0.3019,0.2730,0.5499,0.4423,0.3357,0.0,2.0,1.0,1.0,0.0
17031843800,5,0.777778,41.801657,-87.640476,5,0.8750,0.8369,0.8133,0.0983,0.6732,0.0,0.0,0.0,2.0,0.0
17031843900,5,1.111111,41.778993,-87.576130,5,0.7841,0.5141,0.8993,0.8931,0.8587,0.0,0.0,1.0,1.0,0.0
